In [1]:
import pandas as pd
import numpy as np
import os
from scipy.special import expit

# 저장 경로 확인
if not os.path.exists('data'):
    os.makedirs('data')

n_samples = 10000
np.random.seed(42)

# 기본 데이터셋 구성
data = {
    'age': np.random.randint(18, 70, n_samples),
    'gender': np.random.choice(['Male', 'Female', 'Other'], n_samples),
    'country': np.random.choice(['US', 'UK', 'DE', 'FR', 'CA', 'IN'], n_samples),
    'subscription_type': np.random.choice(['Free', 'Premium', 'Student'], n_samples, p=[0.6, 0.3, 0.1]),
    'device_type': np.random.choice(['Mobile', 'Web', 'Desktop'], n_samples),
    'listening_time': np.random.normal(60, 20, n_samples),
    'songs_played_per_day': np.random.normal(20, 10, n_samples),
    'skip_rate': np.random.beta(2, 5, n_samples),
    'ads_listened_per_week': np.random.randint(0, 30, n_samples),
    'offline_listening': np.random.choice([0, 1], n_samples)
}

df_new = pd.DataFrame(data)

# 이탈 점수 계산
# 단순 합산보다는 조건에 따른 가중치를 확실하게 부여해서 패턴을 만듦
churn_score = np.zeros(n_samples)

# 1. 광고 피로도: 무료 유저가 광고를 너무 많이 들으면 이탈 확률 급증
mask_ad_fatigue = (df_new['subscription_type'] == 'Free') & (df_new['ads_listened_per_week'] > 15)
churn_score[mask_ad_fatigue] += 8.0 

# 2. 콘텐츠 불만족: 노래를 자꾸 넘기는데 청취 시간도 짧은 경우
mask_dislike = (df_new['skip_rate'] > 0.6) & (df_new['listening_time'] < 30)
churn_score[mask_dislike] += 5.0

# 3. 충성 유저: 프리미엄을 쓰거나 오프라인 기능을 잘 활용하는 사람은 잘 안 떠남
mask_loyal = (df_new['subscription_type'] == 'Premium') | (df_new['offline_listening'] == 1)
churn_score[mask_loyal] -= 10.0

# 4. 기본적인 선형 관계 반영 (스킵률이 높을수록 이탈 가능성 높음)
churn_score += (df_new['skip_rate'] * 3.0)
churn_score -= (df_new['songs_played_per_day'] * 0.05)

# 점수를 0~1 사이 확률로 변환 (Sigmoid)
# x값을 2배로 튀겨서 0과 1 양끝으로 확실하게 밀어줌
prob = expit(churn_score * 2.0)

# 이탈 여부 결정 (Labeling)
# 노이즈를 최소화하기 위해 확률이 뚜렷하면 확정 짓고, 애매할 때만 랜덤
rand_vals = np.random.random(n_samples)
df_new['is_churned'] = [
    1 if p > 0.8 else (0 if p < 0.2 else (1 if p > r else 0)) 
    for p, r in zip(prob, rand_vals)
]

# 식별자 추가 및 저장
df_new.insert(0, 'user_id', range(1000, 1000 + n_samples))

save_path = 'data/spotify_churn_dataset.csv'
df_new.to_csv(save_path, index=False)

print(f"데이터 생성 완료: {save_path}")
print("패턴이 명확해져서 모델 학습 시 점수가 잘 나올 겁니다.")

데이터 생성 완료: data/spotify_churn_dataset.csv
패턴이 명확해져서 모델 학습 시 점수가 잘 나올 겁니다.
